In [1]:
si_id = [1,6,4,2,5,3,7,8,31,32,33,34,35,36,37,38,39]

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  서울특별시교육청 정독도서관
서울 종로구
서울특별시 종로구 북촌로5길 48


no 2 :  춘원당한의약박물관
서울 종로구
서울특별시 종로구 돈화문로9길 27


no 3 :  동림매듭박물관
서울 종로구
서울특별시 종로구 북촌로12길 10


no 4 :  북촌문화센터
서울 종로구
서울특별시 종로구 계동길 37


no 5 :  환기미술관
서울 종로구
서울특별시 종로구 자하문로40길 63


no 6 :  목인박물관 목석원
서울 종로구
서울특별시 종로구 창의문로5길 46-1


no 7 :  도산안창호기념관
서울 강남구
서울특별시 강남구 도산대로45길 20


no 8 :  용산공예관
서울 용산구
서울특별시 용산구 이태원로 274


no 9 :  서대문자연사박물관
서울 서대문구
서울특별시 서대문구 연희로32길 51


no 10 :  매헌 윤봉길의사 기념관
서울 서초구
서울특별시 서초구 매헌로 99


no 11 :  부산 트릭아이뮤지엄
부산 중구
부산광역시 중구 대청로126번길 12


no 12 :  복천박물관(부산)
부산 동래구
부산광역시 동래구 복천로 63


no 13 :  부산문화회관
부산 남구
부산광역시 남구 유엔평화로76번길 1


no 14 :  유엔평화기념관
부산 남구
부산광역시 남구 홍곡로320번길 106


no 15 :  부산시립미술관
부산 해운대구
부산광역시 해운대구 APEC로 58


no 16 :  국립부산국악원
부산 부산진구
부산광역시 부산진구 국악로 2


no 17 :  북구문화빙상센터 공연장
부산 북구
부산광역시 북구 금곡대로46번길 50


no 18 :  부산시민회관
부산 동구
부산광역시 동구 자성로133번길 16


no 19 :  요산문학관
부산 금정구
부산광역시 금정구 팔송로 60-6


no 20 :  부산 해운대도서관
부산 해운대구
부산광역시 해운대구 양운로 183


no 21 :  대구예술발전소
대구 중구
대구광역시 중구 달성로22길 31-12


no 22 :  향촌문화관
대구 중구
대구광역시

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="contents"]/div[2]/div[1]/ul/li[9]/div[2]/div/a"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(8,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  수원시립아이파크미술관
경기 수원시
경기도 수원시 팔달구 정조로 833


no 2 :  국립수목원 산림박물관
경기 포천시
경기도 포천시 소흘읍 광릉수목원로 415


no 3 :  경기과천교육도서관
경기 
경기도 과천시 도서관길 36


no 4 :  성남문화원
경기 성남시
경기도 성남시 수정구 산성대로215번길 7


no 5 :  국립여성사전시관
경기 고양시
경기도 고양시 덕양구 화중로104번길 50


no 6 :  광명업사이클아트센터
경기 광명시
경기도 광명시 가학로85번길 142


no 7 :  박물관 얼굴
경기 광주시
경기도 광주시 남종면 분원길 3-6


no 8 :  부천시립북부도서관
경기 부천시
경기도 부천시 삼작로301번길 5


no 9 :  군포시문화예술회관
경기 군포시
경기도 군포시 고산로 599


no 10 :  부천시립꿈빛도서관
경기 부천시
경기도 부천시 신흥로275번길 19


no 11 :  동해문화원
강원 동해시
강원도 동해시 천곡2길 8


no 12 :  영월곤충박물관
강원 영월군
강원도 영월군 영월읍 동강로 716


no 13 :  태백시 문화예술회관
강원 태백시
강원도 태백시 고원로 203


no 14 :  태백 고생대자연사박물관
강원 태백시
강원도 태백시 태백로 2249


no 15 :  축제극장 몸짓
강원 춘천시
강원도 춘천시 춘천로 112


no 16 :  춘천미술관
강원 춘천시
강원도 춘천시 서부대성로 71


no 17 :  동강사진박물관
강원 영월군
강원도 영월군 영월읍 영월로 1909-10


no 18 :  정선아리랑센터(아리랑박물관)
강원 정선군
강원도 정선군 정선읍 애산로 51


no 19 :  춘천문화예술회관
강원 춘천시
강원도 춘천시 효자상길5번길 13


no 20 :  단종역사관
강원 영월군
강원도 영월군 영월읍 단종로 190


no 21 :  청주시한국공예관
충북 청주시
충청북도 청주시 청원구 상당로 314


no 22 :  옥천문화원
충북 옥천군
충청북도 옥천군 

AttributeError: 'NoneType' object has no attribute 'find'

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(12,13):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  안동문화예술의전당
경북 안동시
경상북도 안동시 축제장길 66


no 2 :  테디베어뮤지엄 경주
경북 경주시
경상북도 경주시 보문로 280-34


no 3 :  구미문화원
경북 구미시
경상북도 구미시 선산읍 선산중앙로 83-1


no 4 :  국립경주박물관
경북 경주시
경상북도 경주시 일정로 186


no 5 :  경상북도독립운동기념관
경북 안동시
경상북도 안동시 임하면 독립기념관길 2


no 6 :  삼성현 역사문화관
경북 경산시
경상북도 경산시 남산면 상대로 883-30


no 7 :  경주예술의전당
경북 경주시
경상북도 경주시 알천북로 1


no 8 :  인삼박물관
경북 영주시
경상북도 영주시 풍기읍 죽령로 1378


no 9 :  전통문화콘텐츠박물관
경북 안동시
경상북도 안동시 서동문로 203


no 10 :  칠곡호국평화기념관
경북 칠곡군
경상북도 칠곡군 석적읍 강변대로 1580




총 소요시간은 136.0 초 입니다 
파일 저장 완료: txt 파일명 : c:\temp\2020-09-16-03-32-50-문화시설\2020-09-16-03-32-50-문화시설.txt 
파일 저장 완료: csv 파일명 : c:\temp\2020-09-16-03-32-50-문화시설\2020-09-16-03-32-50-문화시설.csv 
파일 저장 완료: xls 파일명 : c:\temp\2020-09-16-03-32-50-문화시설\2020-09-16-03-32-50-문화시설.xls 


In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(13,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)

    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]

        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  김해문화의전당
경남 김해시
경상남도 김해시 김해대로 2060


no 2 :  고성탈박물관
경남 고성군
경상남도 고성군 고성읍 율대2길 23


no 3 :  고성공룡박물관
경남 고성군
경상남도 고성군 하이면 자란만로 618


no 4 :  하동야생차박물관
경남 하동군
경상남도 하동군 화개면 쌍계로 571-25


no 5 :  창원역사민속관
경남 창원시
경상남도 창원시 의창구 창이대로397번길 25


no 6 :  진해문화센터
경남 창원시
경상남도 창원시 진해구 진해대로 325


no 7 :  성산아트홀
경남 창원시
경상남도 창원시 의창구 중앙대로 181


no 8 :  3·15 아트센터
경남 창원시
경상남도 창원시 마산회원구 삼호로 135


no 9 :  창원의 집
경남 창원시
경상남도 창원시 의창구 사림로16번길 59


no 10 :  한국궁중꽃박물관
경남 양산시
경상남도 양산시 매곡외산로 232


no 11 :  고창고인돌박물관
전북 고창군
전라북도 고창군 고창읍 고인돌공원길 74


no 12 :  남원시립김병종미술관
전북 남원시
전라북도 남원시 함파우길 65-14


no 13 :  군산근대역사박물관
전북 군산시
전라북도 군산시 해망로 240


no 14 :  채만식 문학관
전북 군산시
전라북도 군산시 강변로 449


no 15 :  부안청자박물관
전북 부안군
전라북도 부안군 보안면 청자로 1493


no 16 :  마동도서관
전북 익산시
전라북도 익산시 선화1로 99


no 17 :  무주곤충박물관
전북 무주군
전라북도 무주군 설천면 무설로 1324


no 18 :  국립익산박물관
전북 익산시
전라북도 익산시 금마면 미륵사지로 362


no 19 :  전주역사박물관
전북 전주시
전라북도 전주시 완산구 쑥고개로 259


no 20 :  농업과학관
전북 전주시
전라북도 전주시 완산구 농생명로 300


no 21 :  영광노을전시관
전남 영광군
전라남도 영광군 백수읍 해안로 957


no 22 :  순천만 자연